In [21]:
from pathlib import Path
import os
this_directory = Path(os.getcwd()).resolve()
project_directory = this_directory.parent
import sys
sys.path.append(project_directory.as_posix())
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession\
        .builder\
        .appName("test")\
        .getOrCreate()
from random import random
from operator import add
from pyspark.sql.types import *
schema = StructType([
 StructField("index", IntegerType(), False),
 StructField("in_time", TimestampType(), False),
 StructField("out_time", TimestampType(), False),
 StructField("berthage", IntegerType(), False),
 StructField("section", StringType(), False),
 StructField("admin_region", StringType(), False), 
 StructField("parking_time", LongType(), False),
 ])
data_path = (project_directory/"data/clean_parking_data.csv").as_posix()
df = spark.read.csv(data_path,
                    header=True, 
                    inferSchema=True,
                    
                    )
                #     schema=schema)
df.limit(5).show(5)

+-----+-------------------+-------------------+--------+----------------+------------+------------+
|index|            in_time|           out_time|berthage|         section|admin_region|parking_time|
+-----+-------------------+-------------------+--------+----------------+------------+------------+
|    0|2018-09-01 10:10:00|2018-09-01 12:00:00|  201091|荔园路(蛇口西段)|      南山区|        6600|
|    1|2018-09-01 13:43:35|2018-09-01 14:29:35|  201091|荔园路(蛇口西段)|      南山区|        2760|
|    2|2018-09-01 15:10:54|2018-09-01 16:08:54|  201091|荔园路(蛇口西段)|      南山区|        3480|
|    3|2018-09-01 16:34:03|2018-09-01 17:56:03|  201091|荔园路(蛇口西段)|      南山区|        4920|
|    4|2018-09-01 18:40:20|2018-09-01 20:00:20|  201091|荔园路(蛇口西段)|      南山区|        4800|
+-----+-------------------+-------------------+--------+----------------+------------+------------+



In [25]:
from pyspark.sql.functions import col, hour, minute, floor, explode, count, countDistinct, sum
from pyspark.sql.types import FloatType
# 重新计算停车时间，以分钟为单位
parking_time = 'parking_time(m)'
df = df.withColumn(parking_time, (col('out_time').cast('long')
                                    - col('in_time').cast('long')) / 60)
df.select(parking_time, 'parking_time').show(5)

+---------------+------------+
|parking_time(m)|parking_time|
+---------------+------------+
|          110.0|        6600|
|           46.0|        2760|
|           58.0|        3480|
|           82.0|        4920|
|           80.0|        4800|
+---------------+------------+
only showing top 5 rows



In [27]:
# 以半小时为单位划分时间区间
df = df.withColumn('half_hour', floor((hour(col('in_time')) * 60
                                       + minute(col('in_time'))) / 30))
df.select('in_time', 'half_hour').show(5) # 得到了开始时间属于第几个半小时

+-------------------+---------+
|            in_time|half_hour|
+-------------------+---------+
|2018-09-01 10:10:00|       20|
|2018-09-01 13:43:35|       27|
|2018-09-01 15:10:54|       30|
|2018-09-01 16:34:03|       33|
|2018-09-01 18:40:20|       37|
+-------------------+---------+
only showing top 5 rows



In [29]:
# 按照section、half_hour、berthage进行分组统计
agg_df = df.groupBy('section', 'half_hour', 'berthage').agg(
    countDistinct('berthage').alias('N_s'),
    sum((col(parking_time) >= col('half_hour') * 30) &
        (col(parking_time) < (col('half_hour') + 1) * 30)).alias('N_si')
)
# agg_df.show(5)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "sum(((parking_time(m) >= (half_hour * 30)) AND (parking_time(m) < ((half_hour + 1) * 30))))" due to data type mismatch: Parameter 1 requires the "NUMERIC" or "ANSI INTERVAL" type, however "((parking_time(m) >= (half_hour * 30)) AND (parking_time(m) < ((half_hour + 1) * 30)))" has the type "BOOLEAN".;
'Aggregate [section#649, half_hour#857L, berthage#648], [section#649, half_hour#857L, berthage#648, count(distinct berthage#648) AS N_s#901L, sum(((parking_time(m)#811 >= cast((half_hour#857L * cast(30 as bigint)) as double)) AND (parking_time(m)#811 < cast(((half_hour#857L + cast(1 as bigint)) * cast(30 as bigint)) as double)))) AS N_si#903]
+- Project [index#645, in_time#646, out_time#647, berthage#648, section#649, admin_region#650, parking_time#651, parking_time(m)#811, FLOOR((cast(((hour(in_time#646, Some(Asia/Hong_Kong)) * 60) + minute(in_time#646, Some(Asia/Hong_Kong))) as double) / cast(30 as double))) AS 

In [ ]:




# 计算百分比
agg_df = agg_df.withColumn('percentage', agg_df['N_si'] / agg_df['N_s'] * 100)

# 提取时间区间的开始和结束时间
time_df = df.select(floor((hour(col('in_time')) * 60 +
                    minute(col('in_time'))) / 30).alias('half_hour')).distinct()
time_df = time_df.withColumn('start_time', time_df['half_hour'] * 30)
time_df = time_df.withColumn('end_time', (time_df['half_hour'] + 1) * 30)

# 将时间区间与section进行笛卡尔积，与agg_df进行左连接，得到完整的结果
result_df = time_df.crossJoin(df.select('section').distinct()).join(
    agg_df, ['section', 'half_hour'], 'left')
result_df = result_df.select('start_time', 'end_time', 'section',
                             'N_s', 'N_si', 'percentage').orderBy('section', 'start_time')

result_df.show()

input("按回车继续...")
